# Model 3: CNNs-LSTM-with-Aug ⭐ BEST MODEL

## 7-Layer CNN-LSTM Hybrid

**Architecture:** TimeDistributed CNN + LSTM(100) + Dense(2)
- Input: (1, 128, 128, 3) time-distributed
- Epochs: 25, Batch: 16

**Target:** 99.92% accuracy

In [ ]:
%run 00_utils_and_config.ipynb

## 1. Load Data

In [ ]:
X_train = np.load(CONFIG["processed_data_path"] / "X_train_128.npy").astype("float32") / 255.0
X_test = np.load(CONFIG["processed_data_path"] / "X_test_128.npy").astype("float32") / 255.0
y_train = np.load(CONFIG["processed_data_path"] / "y_train.npy")
y_test = np.load(CONFIG["processed_data_path"] / "y_test.npy")

# Reshape for LSTM: (samples, timesteps, height, width, channels)
# Then convert to PyTorch format (samples, timesteps, channels, height, width)
X_train = X_train.reshape((-1, 1, 128, 128, 3))
X_test = X_test.reshape((-1, 1, 128, 128, 3))

# Convert to PyTorch format: (N, T, C, H, W)
X_train = np.transpose(X_train, (0, 1, 4, 2, 3))
X_test = np.transpose(X_test, (0, 1, 4, 2, 3))

print(f"Reshaped: {X_train.shape}, {X_test.shape}")

# Create tensors and dataloaders
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_train_tensor = torch.from_numpy(y_train).long()
y_test_tensor = torch.from_numpy(y_test).long()

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

## 2. Build CNN-LSTM Model

In [ ]:
class CNNLSTM(nn.Module):
    """CNN-LSTM Hybrid for sequential MRI slice processing."""
    
    def __init__(self, input_channels=3, lstm_hidden=100, num_classes=2):
        super(CNNLSTM, self).__init__()
        
        # CNN layers (applied to each timestep)
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # After 128x128 -> 64x64 -> 32x32
        self.flatten_size = 32 * 32 * 32
        
        # LSTM layer
        self.lstm = nn.LSTM(self.flatten_size, lstm_hidden, batch_first=True)
        
        # Output layer
        self.fc = nn.Linear(lstm_hidden, num_classes)
    
    def forward(self, x):
        # x shape: (batch, timesteps, channels, height, width)
        batch_size, timesteps, C, H, W = x.size()
        
        # Process each timestep through CNN
        c_out = []
        for t in range(timesteps):
            # CNN for this timestep
            c = F.relu(self.conv1(x[:, t, :, :, :]))
            c = self.pool1(c)
            c = F.relu(self.conv2(c))
            c = self.pool2(c)
            c = c.view(batch_size, -1)  # Flatten
            c_out.append(c)
        
        # Stack timesteps
        lstm_input = torch.stack(c_out, dim=1)  # (batch, timesteps, features)
        
        # LSTM
        lstm_out, _ = self.lstm(lstm_input)
        
        # Take last timestep output
        last_output = lstm_out[:, -1, :]
        
        # Output
        out = self.fc(last_output)
        return out

model = CNNLSTM(input_channels=3, lstm_hidden=100, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## 3. Train

In [ ]:
start_time = time.time()
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_val_acc = 0.0
best_model_path = str(CONFIG["saved_models_path"] / "model3_cnn_lstm_best.pth")

# Split for validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
train_loader_split = DataLoader(train_subset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=16, shuffle=False)

for epoch in range(25):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, labels in train_loader_split:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / total
    train_acc = correct / total
    
    # Validate
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_loss /= val_total
    val_acc = val_correct / val_total
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Epoch {epoch+1}/25 - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f} ⭐ BEST")
    else:
        print(f"Epoch {epoch+1}/25 - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

training_time = time.time() - start_time
print(f"\n✓ Training complete - Best val acc: {best_val_acc*100:.2f}%")

## 4. Evaluate

In [ ]:
best_model = CNNLSTM(input_channels=3, lstm_hidden=100, num_classes=2).to(device)
best_model.load_state_dict(torch.load(best_model_path))
best_model.eval()

start_time = time.time()
all_preds, all_probs = [], []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = best_model(inputs)
        probs = F.softmax(outputs, dim=1)
        all_probs.append(probs.cpu().numpy())
        _, predicted = torch.max(outputs.data, 1)
        all_preds.append(predicted.cpu().numpy())

y_pred_proba = np.vstack(all_probs)
y_pred = np.concatenate(all_preds)
testing_time = (time.time() - start_time) * 1000

metrics = calculate_all_metrics(y_test, y_pred, y_pred_proba[:, 1])
print_metrics(metrics, "Model 3: CNN-LSTM (BEST)")
print(f"⭐ Accuracy: {metrics['accuracy']*100:.2f}% (Target: 99.92%)")

## 5. Save

In [ ]:
plot_training_history(history, "Model 3: CNN-LSTM", CONFIG["results_path"] / "training_curves" / "model3_training.png")
torch.save(model.state_dict(), CONFIG["saved_models_path"] / "model3_cnn_lstm_final.pth")

results = {"model_name": "CNN-LSTM-with-Aug", "accuracy": float(metrics["accuracy"]), "precision": float(metrics["precision"]), "recall": float(metrics["recall"]), "f1_score": float(metrics["f1_score"]), "specificity": float(metrics["specificity"]), "training_time_seconds": float(training_time), "testing_time_ms": float(testing_time)}

with open(CONFIG["results_path"] / "model3_results.json", "w") as f:
    json.dump(results, f, indent=2)

print(f"\n✓ Model 3 (BEST) saved - Accuracy: {metrics['accuracy']*100:.2f}%")